In [1]:
import pandas as pd
import os
os.chdir(r"C:\Training\Kaggle\Competitions\Playground Competitions\Mohs_Dec23")
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, GridSearchCV, cross_val_score
import numpy as np

In [2]:
train = pd.read_csv("train.csv", index_col=0)
train.head(3)

,allelectrons_Total,density_Total,allelectrons_Average,val_e_Average,atomicweight_Average,ionenergy_Average,el_neg_chi_Average,R_vdw_element_Average,R_cov_element_Average,zaratio_Average,density_Average,Hardness
id,,,,,,,,,,,,
0,100.0,0.841611,10.0,4.8,20.612526,11.08810,2.766,1.732,0.860,0.496070,0.91457,6.0
1,100.0,7.558488,10.0,4.8,20.298893,12.04083,2.755,1.631,0.910,0.492719,0.71760,6.5
2,76.0,8.885992,15.6,5.6,33.739258,12.08630,2.828,1.788,0.864,0.481478,1.50633,2.5


In [3]:
X = train.drop('Hardness', axis=1)
y = train['Hardness']

### Lasso

In [4]:
lasso = Lasso()
params = {'alpha': np.linspace(0.001, 10, 20)}
kfold = KFold(n_splits=5, shuffle=True, random_state=23)
gcv_ls = GridSearchCV(lasso, cv=kfold, param_grid=params)
gcv_ls.fit(X, y)
print(gcv_ls.best_params_)
print(gcv_ls.best_score_)

{'alpha': 0.001}
0.24069752131958472


In [5]:
best_lasso = gcv_ls.best_estimator_

### Ridge

In [6]:
ridge = Ridge()
params = {'alpha': np.linspace(0.001, 10, 20)}
kfold = KFold(n_splits=5, shuffle=True, random_state=23)
gcv_rg = GridSearchCV(ridge, cv=kfold, param_grid=params)
gcv_rg.fit(X, y)
print(gcv_rg.best_params_)
print(gcv_rg.best_score_)

{'alpha': 1.0535263157894736}
0.24085489603847


In [7]:
best_rg = gcv_rg.best_estimator_

### Elastic Net

In [8]:
elastic = ElasticNet()
params = {'alpha': np.linspace(0.001, 10, 20),
         'l1_ratio': np.linspace(0.001, 1, 10)}
gcv_el = GridSearchCV(elastic, cv=kfold, param_grid=params)
gcv_el.fit(X, y)

GridSearchCV(cv=KFold(n_splits=5, random_state=23, shuffle=True),
             estimator=ElasticNet(),
             param_grid={'alpha': array([1.00000000e-03, 5.27263158e-01, 1.05352632e+00, 1.57978947e+00,
       2.10605263e+00, 2.63231579e+00, 3.15857895e+00, 3.68484211e+00,
       4.21110526e+00, 4.73736842e+00, 5.26363158e+00, 5.78989474e+00,
       6.31615789e+00, 6.84242105e+00, 7.36868421e+00, 7.89494737e+00,
       8.42121053e+00, 8.94747368e+00, 9.47373684e+00, 1.00000000e+01]),
                         'l1_ratio': array([0.001, 0.112, 0.223, 0.334, 0.445, 0.556, 0.667, 0.778, 0.889,
       1.   ])})

In [9]:
best_el = gcv_el.best_estimator_

In [10]:
print(gcv_el.best_params_)
print(gcv_el.best_score_)

{'alpha': 0.001, 'l1_ratio': 1.0}
0.24069752131958472


### Inferencing

#### UnLabeled Data

In [24]:
tst = pd.read_csv("test.csv", index_col=0)
tst.head(3)

,allelectrons_Total,density_Total,allelectrons_Average,val_e_Average,atomicweight_Average,ionenergy_Average,el_neg_chi_Average,R_vdw_element_Average,R_cov_element_Average,zaratio_Average,density_Average
id,,,,,,,,,,,
10407,884.0,121.420000,35.36,5.28,82.561240,9.370384,2.298,1.9072,1.200,0.461899,1.79459
10408,90.0,9.931960,18.00,5.60,39.568056,12.086300,2.868,1.6520,0.864,0.476196,1.41194
10409,116.0,7.767992,11.60,4.80,23.231818,11.023840,2.644,1.7940,0.960,0.499514,0.78834


We predict with Ridge Regression object as R2 score for Ridge came out to be the best

In [27]:
predictions = best_rg.predict(tst)
np.sum(predictions < 0)

2

In [28]:
predictions = np.where(predictions<0, 2, predictions)
np.sum(predictions < 0)

0

#### Sample Submission

In [29]:
submit = pd.read_csv("sample_submission.csv")
submit['Hardness'] = predictions
submit.head(3)

,id,Hardness
0,10407,3.483739
1,10408,4.833068
2,10409,5.283257


In [30]:
submit.to_csv("sbt_rg_29Dec.csv", index=False)